In [19]:
sys_prom = """
You are a documentation and language semantics expert. Your job is to breakdown process and data documents furnished by a user and extract only limited and relevant knowledge. You break down the documents into chunks of knowledge. Each one of the chunks is semantically atomic, i.e., its knowledge is limited to one and only one concept. A chunk is shorter than or as long as one sentence. The concept captured by a chunk is simple. A complex concept is broken down into multiple chunks. Chunks can overlap too. They fall completely within the bounds of the information contained in the user provided documents. Given a user query you find out which chunks are most relevant to the query. Relevant chunks include the chunks which may help answer the query and other chunks which hold information on various concepts in the query. You can also include chunks which contain supporting knowledge, i.e., information on the relevant chunks. Order each chunk by relevance. Do not overlook anything that may be relevant.
"""

In [20]:
prom = """
Find out all the chunks of knowledge relevant to the user's query: Which Brand had a significant PoD growth / decline ?
"""

In [5]:
file = "gs://p0s0a31/sao_chatbot/kg/optimization.txt"

In [6]:
temp = 0
topP = 0.95
out_tok = 1024

In [7]:
resp_sch = {
    "type": "OBJECT",
    "properties": {
        "resp": {
            "type": "ARRAY",
            "description": "a list of chunks",
            "items": {
                "description": "a chunk characterized by a rank and text",
                "type": "OBJECT",
                "properties": {
                    "no.": {
                        "type": "INTEGER",
                        "description": "rank based on relevance"
                    },
                    "txt": {
                        "type": "STRING",
                        "description": "the text of the chunk as in the user provided document"
                    }
                },
                "required": [
                    "no.",
                    "txt"
                ]
            },
            "minItems": "0",
            "maxItems": "10"
        }
    },
    "required": [
        "resp"
    ]
}

In [21]:
payload = {
    "contents": [
        {
            "role": "user",
            "parts": [
                {
                    "text": prom
                },
                {
                    "fileData": {
                        "mimeType": "text/plain",
                        "fileUri": file
                    }
                }
            ]
        }
    ]
    , "systemInstruction": {
        "parts": [
        {
            "text": sys_prom
        }
      ]
    }
    , "generationConfig": {
        "responseModalities": ["TEXT"]
        ,"temperature": temp
        ,"maxOutputTokens": out_tok
        ,"topP": topP
        ,"responseMimeType": "application/json"
        ,"responseSchema": resp_sch
    }
}

In [9]:
import google.auth
import google.auth.transport.requests

In [10]:
def load_gcloud_oauth_token():
    cred, proj = google.auth.default() # creds.valid is False, and creds.token is None
    auth_req = google.auth.transport.requests.Request()
    cred.refresh(auth_req) # need to refresh credentials to populate those
    access_token:str = cred.token
    return {"Authorization": f"Bearer {access_token}"}

In [11]:
from requests import request, models

In [12]:
project = "wmt-mtech-assortment-ml-prod"
model = "gemini-1.5-pro-001"
gateway_url = f"https://us-central1-aiplatform.googleapis.com/v1/projects/{project}/locations/us-central1/publishers/google/models/{model}:generateContent"
header = load_gcloud_oauth_token()

In [22]:
response:models.Response = request("POST", 
                                   gateway_url, 
                                   headers=header, 
                                   json=payload)

In [23]:
chunks = response.json()["candidates"][0]["content"]

In [24]:
chunks

{'role': 'model',
 'parts': [{'text': '{"resp": [{"no.": 1, "txt": "PoD change is the number of PoDs added, also called PoD growth, or the number of PoDs deleted, also called PoD decline."}, {"no.": 2, "txt": "PoD change can only be computed at a level of aggregation over PoD."}, {"no.": 3, "txt": "A PoD is an item-store pair."}, {"no.": 4, "txt": "The optimizer tries to recommend high-performing PoDs, i.e., those with higher productivity."}, {"no.": 5, "txt": "The recommendation, also called status, for a roll-up at a store can be an \'add\', \'delete\', \'maintain\' or \'not added\'."}, {"no.": 6, "txt": "An \'add\' recommendation for a PoD means that its new, not present in the current mod, and it should be added to the mod."}, {"no.": 7, "txt": "A \'delete\' recommendation for a PoD means it should be removed from the mod."}, {"no.": 8, "txt": "Merchants decide the universe, i.e., the roll-ups and stores for a relay."}, {"no.": 9, "txt": "A set of PoDs, a subset of this universe is